# North American Rig Count

The North American rig count is a key indicator of drilling activity in oil and gas production in the United States and Canada. It is closely monitored by industry stakeholders to gauge market conditions and forecast trends. This notebook creates the North American rig count tables and generates a brief automated commentary, summarizing the changes in the rig count.

### Structure of the Script

- **Dependencies:** Import all necessary packages and libraries to run the code.
- **Input Parameters:** Gather and prepare all data required for table creation.
- **Tables:** Create the weekly rig count tables for the US and Canada.
- **Commentary:** Write a short commentary, summarizing the weekly changes in the rig count.

### Dependencies

In [1]:
# Standard packages
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from io import BytesIO
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore') # To ignore warnings for log

### Input Parameters

The code below fetches the latest North America rig count report from the Baker Hughes website by scraping the page for a specific report link. It then downloads the linked Excel file, reads it into an in-memory Pandas DataFrame, and prints the first few rows of the data.

In [2]:
# Fetching the rig count report from the Baker Hughes website and writing it into in-memory stream.
url = "https://bakerhughesrigcount.gcs-web.com/na-rig-count"
response =  requests.get(url)

# Searching the parsed html ('soup') for a tag whose text matches 'North America Rig Count Report - New Report' on the NA rig count page
soup = BeautifulSoup(response.text, 'html.parser')
link = soup.find('a', string=re.compile('North America Rig Count Report - New Report'))

# Checking if the link is not none and extracting the URL from the 'href' attribute of the 'link'
if link:
    file_url = link['href']
    if not file_url.startswith('http'):
        file_url = 'https://bakerhughesrigcount.gcs-web.com' + file_url

    excel_data = BytesIO(requests.get(file_url).content) # Creating an in-memory binary stream from the content of the downloaded file
    RigCount = pd.read_excel(excel_data, sheet_name='NAM Weekly', skiprows=10) # Reading the excel file from the in-memory binary stream ('excel_data') into a Pandas DataFrame
else:
    raise ValueError('File link not found.')

print(RigCount.head())

  Country                County  Basin GOM DrillFor Location State/Province  \
0  CANADA       FOOTHILLS FRONT  Other  No      Gas     Land        ALBERTA   
1  CANADA  NORTHWESTERN ALBERTA  Other  No      Gas     Land        ALBERTA   
2  CANADA  SOUTHEASTERN ALBERTA  Other  No      Gas     Land        ALBERTA   
3  CANADA       CENTRAL ALBERTA  Other  No      Oil     Land        ALBERTA   
4  CANADA  EAST CENTRAL ALBERTA  Other  No      Oil     Land        ALBERTA   

    Trajectory  Year  Month US_PublishDate  Rig Count Value  
0  Directional  2012      1     2012-01-06               24  
1  Directional  2012      1     2012-01-06                1  
2  Directional  2012      1     2012-01-06                3  
3  Directional  2012      1     2012-01-06                1  
4  Directional  2012      1     2012-01-06                1  


#### Relevant Dates

Baker Hughes releases rig count data on the last business day of each week, which is usually Friday. However, during certain holidays such as Black Friday, Good Friday, or Christmas, the release may occur on Thursday or Wednesday. The script below accounts for these exceptions and accurately determines the publication dates for the rig count reports.

In [3]:
# Function to determine the last four Fridays and their preceding Thursdays and Wednesdays
def get_report_days():
    today = datetime.now() # Retrieves the current date and time and assigns it to the variable 'today'
    weekdays = [] # An empty list to store the dates of the last four rig count reporting days
    report_day_count = 0 # A counter initialized to 0 to keep track of how many rig count reporting days have been found
    # Iterate backwards day by day from today
    while report_day_count < 4: # A loop that continues until last four rig count reporting days have been found
        if today.weekday() == 4: # If today is Friday
            report_day_count += 1 # Increment the rig count day count by 1
            weekdays.append(today.date()) # Adds 'today' the 'weekdays' list
        today -= timedelta(days=1) # If 'today' is not Friday and rig count data is not released today, then substract one day from 'today' to move to the previous day

    # Remove duplicates and add preceding Thursdays or Wednesdays
    unique_weekdays = []
    added_dates = set() # A set to keep track of already added dates for quick lookup

    for day in weekdays: # Iterate through each day in the 'weekdays' list
        if day.weekday() == 4 and day not in added_dates: # Checks if the 'day' is a Friday and not already added
            unique_weekdays.append(day) # If so, the it adds the 'day' to the 'unique_weekdays' list and the 'added_dates' set
            added_dates.add(day)
        elif day.weekday() == 3 and day not in added_dates and (day + timedelta(days=1)) not in added_dates: # Checks if the 'day' is a Thursday and not already added to added_set, and ensures that the next day is not already added in the 'added_dates' set either.
            unique_weekdays.append(day) # If all the conditions above are met, then the 'day' is added to 'unique_weekdays' and 'added_dates', ensuring that a Thursday is only added if no Friday has been added for that week.
            added_dates.add(day)
        elif day.weekday() == 2 and day not in added_dates and (day + timedelta(days=1)) not in added_dates and (day + timedelta(days=2)) not in added_dates:
            unique_weekdays.append(day)
            added_dates.add(day)
    
    return unique_weekdays[:4] # Slices the 'unique_weekdays' list to return only the last 4 dates.
            
last_four_report_days = get_report_days()

# Format the dates as a list of strings
formatted_dates = [day.strftime('%m-%d-%Y') for day in last_four_report_days]

# Print the last four rig count reporting days
print('\nLast Four Rig Count Reporting Days:')
for date in formatted_dates:
    print(date)


Last Four Rig Count Reporting Days:
09-06-2024
08-30-2024
08-23-2024
08-16-2024


### Tables

The code filters and processes rig count data from the last four reporting days by setting the date as the index and formatting state/province/basin names. It then uses a function to group the data by specified columns, calculate totals, and sort it accordingly. Finally, it creates and displays separate DataFrames for US and Canada rig counts based on their type (Gas, Oil, Miscellaneous).

In [4]:
# Filter the DataFrame for the last four reporting days
filtered_RigCount = RigCount[RigCount['US_PublishDate'].isin(last_four_report_days)]

# Set US_PublishDate as the index and format it
filtered_RigCount.set_index('US_PublishDate', inplace=True)

# Convert State/Province and Country names to title case
filtered_RigCount['State/Province'] = filtered_RigCount['State/Province'].str.title()
filtered_RigCount['Country'] = filtered_RigCount['Country'].str.title()

# Function to process the filtered DataFrame based on given condition
def process_dataframe(df, condition, groupby_columns, display_columns, sort_columns):
    filtered_df = df[condition].copy()
    filtered_df['index'] = filtered_df.index

    # Group by and sum
    grouped_df = filtered_df.groupby(groupby_columns + ['index'])['Rig Count Value'].sum().reset_index()
    grouped_df.set_index('index', inplace=True)
    
    # Create 'Total' rows
    total_rows = grouped_df.groupby(level=0).sum().reset_index()
    for col in groupby_columns:
        if col == 'State/Province' or col == 'Basin':
            total_rows[col] = 'Total'
        else:
            total_rows[col] = grouped_df[col].iloc[0]  # Keep the original value for other columns
    total_rows.set_index('index', inplace=True)

    # Combine data and totals
    combined_df = pd.concat([grouped_df, total_rows])
    combined_df.sort_values(by=['index'] + sort_columns, ascending=[False] + [True] * len(sort_columns), inplace=True)
    
    return combined_df[display_columns]

# Creating DataFrames based on specified conditions and processing them
us_gas_rig_count = process_dataframe(
    filtered_RigCount, 
    (filtered_RigCount['Country'] == 'United States') & 
    (filtered_RigCount['DrillFor'] == 'Gas'),
    ['Country', 'DrillFor', 'State/Province'],
    ['Country', 'DrillFor', 'State/Province', 'Rig Count Value'],
    ['State/Province']
)

us_oil_rig_count = process_dataframe(
    filtered_RigCount, 
    (filtered_RigCount['Country'] == 'United States') & 
    (filtered_RigCount['DrillFor'] == 'Oil'),
    ['Country', 'DrillFor', 'State/Province'],
    ['Country', 'DrillFor', 'State/Province', 'Rig Count Value'],
    ['State/Province']
)

us_misc_rig_count = process_dataframe(
    filtered_RigCount, 
    (filtered_RigCount['Country'] == 'United States') & 
    (filtered_RigCount['DrillFor'] == 'Miscellaneous'),
    ['Country', 'DrillFor', 'State/Province'],
    ['Country', 'DrillFor', 'State/Province', 'Rig Count Value'],
    ['State/Province']
)

us_basin_rig_count = process_dataframe(
    filtered_RigCount, 
    (filtered_RigCount['Country'] == 'United States') & 
    (filtered_RigCount['DrillFor'].isin(['Gas', 'Oil', 'Miscellaneous'])),
    ['Country', 'Basin'],
    ['Country', 'Basin', 'Rig Count Value'],
    ['Basin']
)

can_gas_rig_count = process_dataframe(
    filtered_RigCount, 
    (filtered_RigCount['Country'] == 'Canada') & 
    (filtered_RigCount['DrillFor'] == 'Gas'),
    ['Country', 'DrillFor', 'State/Province'],
    ['Country', 'DrillFor', 'State/Province', 'Rig Count Value'],
    ['State/Province']
)

can_oil_rig_count = process_dataframe(
    filtered_RigCount, 
    (filtered_RigCount['Country'] == 'Canada') & 
    (filtered_RigCount['DrillFor'] == 'Oil'),
    ['Country', 'DrillFor', 'State/Province'],
    ['Country', 'DrillFor', 'State/Province', 'Rig Count Value'],
    ['State/Province']
)

can_misc_rig_count = process_dataframe(
    filtered_RigCount, 
    (filtered_RigCount['Country'] == 'Canada') & 
    (filtered_RigCount['DrillFor'] == 'Miscellaneous'),
    ['Country', 'DrillFor', 'State/Province'],
    ['Country', 'DrillFor', 'State/Province', 'Rig Count Value'],
    ['State/Province']
)

# Display the resulting DataFrames
print("US Gas Rig Count:\n", us_gas_rig_count)
print("\nUS Oil Rig Count:\n", us_oil_rig_count)
print("\nUS Miscellaneous Rig Count:\n", us_misc_rig_count)
print("\nUS Basin Rig Count:\n", us_basin_rig_count)
print("\nCanada Gas Rig Count:\n", can_gas_rig_count)
print("\nCanada Oil Rig Count:\n", can_oil_rig_count)
print("\nCanada Miscellaneous Rig Count:\n", can_misc_rig_count)

US Gas Rig Count:
                   Country DrillFor State/Province  Rig Count Value
index                                                             
2024-09-06  United States      Gas     California                2
2024-09-06  United States      Gas       Colorado                4
2024-09-06  United States      Gas      Louisiana               24
2024-09-06  United States      Gas       Michigan                1
2024-09-06  United States      Gas     New Mexico                5
2024-09-06  United States      Gas           Ohio                7
2024-09-06  United States      Gas   Pennsylvania               16
2024-09-06  United States      Gas          Texas               22
2024-09-06  United States      Gas          Total               94
2024-09-06  United States      Gas           Utah                2
2024-09-06  United States      Gas  West Virginia                8
2024-09-06  United States      Gas        Wyoming                3
2024-08-30  United States      Gas     Cali

The code defines a function to reformat DataFrames by pivoting the data with dates as columns, sorting dates in ascending order, and adjusting column names. It then creates display tables for various categories (US Gas, Oil, Miscellaneous, Basin, and Canadian Gas, Oil, Miscellaneous) using this function. Finally, it prints these reformatted tables.

In [5]:
# Function to reformat the DataFrames for display with dates sorted in ascending order and specific column adjustments
def create_display_table(df, group_column, display_name, date_format='%b-%d'):
    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)
    
    # Pivot the data to have dates as columns
    pivot_df = df.pivot_table(
        index=group_column, 
        columns=df.index.strftime(date_format), 
        values='Rig Count Value', 
        aggfunc='sum',
        fill_value=0
    )
    # Reset index to have the group_column as a regular column and rename it
    pivot_df.reset_index(inplace=True)
    pivot_df.rename(columns={group_column: display_name}, inplace=True)
    
    # Reorder the columns to place the display_name (State, Province, Basin) first
    column_order = [display_name] + sorted(pivot_df.columns[1:], key=lambda x: pd.to_datetime(x, format=date_format))
    pivot_df = pivot_df[column_order]

    # Move the "Total" row to the bottom if it exists
    if 'Total' in pivot_df[display_name].values:
        total_row = pivot_df[pivot_df[display_name] == 'Total']
        pivot_df = pivot_df[pivot_df[display_name] != 'Total']
        pivot_df = pd.concat([pivot_df, total_row], ignore_index=True)
    
    return pivot_df

# Create display tables for US (Gas, Oil, Miscellaneous) and Canada (Gas, Oil, Miscellaneous), and Basin (US)
us_gas_display_table = create_display_table(us_gas_rig_count, 'State/Province', 'State')
us_oil_display_table = create_display_table(us_oil_rig_count, 'State/Province', 'State')
us_misc_display_table = create_display_table(us_misc_rig_count, 'State/Province', 'State')
us_basin_display_table = create_display_table(us_basin_rig_count, 'Basin', 'Basin')
can_gas_display_table = create_display_table(can_gas_rig_count, 'State/Province', 'Province')
can_oil_display_table = create_display_table(can_oil_rig_count, 'State/Province', 'Province')
can_misc_display_table = create_display_table(can_misc_rig_count, 'State/Province', 'Province')

# Display the resulting tables without the index column
print("US Gas Rig Count Table:\n", us_gas_display_table.to_string(index=False))
print("\nUS Oil Rig Count Table:\n", us_oil_display_table.to_string(index=False))
print("\nUS Miscellaneous Rig Count Table:\n", us_misc_display_table.to_string(index=False))
print("\nUS Basin Rig Count Table:\n", us_basin_display_table.to_string(index=False))
print("\nCanada Gas Rig Count Table:\n", can_gas_display_table.to_string(index=False))
print("\nCanada Oil Rig Count Table:\n", can_oil_display_table.to_string(index=False))
print("\nCanada Miscellaneous Rig Count Table:\n", can_misc_display_table.to_string(index=False))

US Gas Rig Count Table:
         State  Aug-16  Aug-23  Aug-30  Sep-06
   California       2       2       2       2
     Colorado       4       4       4       4
    Louisiana      23      24      24      24
     Michigan       1       1       1       1
   New Mexico       5       5       5       5
         Ohio       7       7       7       7
 Pennsylvania      21      21      18      16
        Texas      24      23      23      22
         Utah       3       2       2       2
West Virginia       5       5       6       8
      Wyoming       3       3       3       3
        Total      98      97      95      94

US Oil Rig Count Table:
        State  Aug-16  Aug-23  Aug-30  Sep-06
      Alaska      10      10      10      10
  California       4       4       4       4
    Colorado      10      10       9       8
   Louisiana      13      13      14      14
     Montana       1       1       1       1
  New Mexico     102     101     100     101
North Dakota      35      33      33

### Create Weekly Change Tables

The code defines a function to generate a weekly change table from a DataFrame by comparing the latest and previous weeks' data. It calculates changes where data is available and optionally renames the location column for clarity. The function is then applied to various rig count categories, and the results are displayed.

In [14]:
def create_weekly_change_table(df, display_name, rename_column_to=None):
    # Determine the column name for location (either 'State' or 'Province')
    location_column = df.columns[0]  # This should be 'State' or 'Province'
    
    # Sort the columns to ensure the dates are in the right order
    sorted_columns = sorted(df.columns[1:], key=lambda x: pd.to_datetime(x, format='%b-%d'))

    # Check if there are at least two weeks of data
    if len(sorted_columns) < 2:
        # If there's only one week of data, create a change table from 0 to current week
        this_week_col = sorted_columns[-1]
        change_df = df[[location_column, this_week_col]].copy()
        change_df['Last Week'] = 0  # Assume no rigs last week if no data exists
        change_df['This Week'] = change_df[this_week_col]
        change_df['Change'] = change_df['This Week'] - change_df['Last Week']

        # Filter rows where there is no change, except for the "Total" row
        change_df = change_df[(change_df['Change'] != 0) | (change_df[location_column] == 'Total')]

        # Rename 'State' or 'Province' column if required
        if rename_column_to:
            change_df.rename(columns={location_column: rename_column_to}, inplace=True)
        
        return change_df[[location_column, 'Last Week', 'This Week', 'Change']]  # Return the change table

    # If at least two weeks of data exist, calculate the difference normally
    last_week_col = sorted_columns[-2]
    this_week_col = sorted_columns[-1]

    # Create a DataFrame with Last Week, This Week, and Change columns
    change_df = df[[location_column, last_week_col, this_week_col]].copy()

    # Fill missing values in the 'Last Week' column with 0 to account for no rigs last week
    change_df[last_week_col] = change_df[last_week_col].fillna(0)

    # Fill missing values in the 'This Week' column with 0 to handle current missing data (if necessary)
    change_df[this_week_col] = change_df[this_week_col].fillna(0)

    # Rename columns for clarity
    change_df.columns = [location_column, 'Last Week', 'This Week']

    # Calculate the change
    change_df['Change'] = change_df['This Week'] - change_df['Last Week']

    # Filter out rows where there is no change, except for the "Total" row
    change_df = change_df[(change_df['Change'] != 0) | (change_df[location_column] == 'Total')]

    # Rename 'State' or 'Province' column if required
    if rename_column_to:
        change_df.rename(columns={location_column: rename_column_to}, inplace=True)

    return change_df[[location_column, 'Last Week', 'This Week', 'Change']]

# Create weekly change tables for each category separately with appropriate renaming
us_gas_change_table = create_weekly_change_table(us_gas_display_table, 'State')
us_oil_change_table = create_weekly_change_table(us_oil_display_table, 'State')
us_misc_change_table = create_weekly_change_table(us_misc_display_table, 'State')
us_basin_change_table = create_weekly_change_table(us_basin_display_table, 'Basin', rename_column_to='Basin')

can_gas_change_table = create_weekly_change_table(can_gas_display_table, 'Province', rename_column_to='Province')
can_oil_change_table = create_weekly_change_table(can_oil_display_table, 'Province', rename_column_to='Province')
can_misc_change_table = create_weekly_change_table(can_misc_display_table, 'Province', rename_column_to='Province')

# Display the resulting weekly change tables if not empty
if not us_gas_change_table.empty:
    print("US Gas Weekly Change Table:\n", us_gas_change_table.to_string(index=False))
if not us_oil_change_table.empty:
    print("\nUS Oil Weekly Change Table:\n", us_oil_change_table.to_string(index=False))
if not us_misc_change_table.empty:
    print("\nUS Miscellaneous Weekly Change Table:\n", us_misc_change_table.to_string(index=False))
if not us_basin_change_table.empty:
    print("\nUS Basin Weekly Change Table:\n", us_basin_change_table.to_string(index=False))

if not can_gas_change_table.empty:
    print("\nCanada Gas Weekly Change Table:\n", can_gas_change_table.to_string(index=False))
if not can_oil_change_table.empty:
    print("\nCanada Oil Weekly Change Table:\n", can_oil_change_table.to_string(index=False))
if not can_misc_change_table.empty:
    print("\nCanada Miscellaneous Weekly Change Table:\n", can_misc_change_table.to_string(index=False))

US Gas Weekly Change Table:
         State  Last Week  This Week  Change
 Pennsylvania         18         16      -2
        Texas         23         22      -1
West Virginia          6          8       2
        Total         95         94      -1

US Oil Weekly Change Table:
      State  Last Week  This Week  Change
  Colorado          9          8      -1
New Mexico        100        101       1
  Oklahoma         39         38      -1
     Texas        251        252       1
     Total        483        483       0

US Miscellaneous Weekly Change Table:
 State  Last Week  This Week  Change
Total          5          5       0

US Basin Weekly Change Table:
       Basin  Last Week  This Week  Change
DJ-Niobrara          9          8      -1
      Other         97         96      -1
    Permian        305        306       1
      Total        583        582      -1

Canada Gas Weekly Change Table:
         Province  Last Week  This Week  Change
         Alberta         45         47  

### Commentary

The code generates a weekly commentary on natural gas and oil rig counts based on data from the US. It calculates and describes the total rig count changes for both gas and oil, identifies states with increases or decreases in gas rigs, and formats this information into a readable paragraph.

In [25]:
from datetime import datetime

def generate_paragraph(formatted_dates, gas_df, oil_df):
    # Convert the latest date to the "Sep-6" format
    latest_date_raw = formatted_dates[0]
    latest_date = datetime.strptime(latest_date_raw, '%m-%d-%Y').strftime('%b-%d')

    # Extract total rig count for "This Week" from US Gas and US Oil DataFrames
    try:
        gas_total_latest = gas_df[gas_df['State'] == 'Total']['This Week'].values[0]
        oil_total_latest = oil_df[oil_df['State'] == 'Total']['This Week'].values[0]

        # Extract the "Total" row's change for gas and oil
        gas_change_sum = gas_df[gas_df['State'] == 'Total']['Change'].values[0]
        oil_change_sum = oil_df[oil_df['State'] == 'Total']['Change'].values[0]
    except IndexError:
        return "Error: 'Total' row not found in one of the DataFrames."

    # Determine the change phrases for gas and oil
    if gas_change_sum > 0:
        gas_rig_word = "rig" if gas_change_sum == 1 else "rigs"
        gas_change_phrase = f"stood at {gas_total_latest}, reflecting an increase of {gas_change_sum} {gas_rig_word}"
    elif gas_change_sum < 0:
        gas_rig_word = "rig" if gas_change_sum == -1 else "rigs"
        gas_change_phrase = f"stood at {gas_total_latest}, reflecting a decrease of {abs(gas_change_sum)} {gas_rig_word}"
    else:
        gas_change_phrase = f"remained unchanged at {gas_total_latest}"

    if oil_change_sum > 0:
        oil_change_phrase = f"representing an increase of {oil_change_sum} rig{'s' if oil_change_sum > 1 else ''}, WoW"
    elif oil_change_sum < 0:
        oil_change_phrase = f"representing a decrease of {abs(oil_change_sum)} rig{'s' if oil_change_sum < -1 else ''}, WoW"
    else:
        oil_change_phrase = "flat, WoW"

    # List of states with positive and negative changes for gas, excluding "Total"
    positive_changes_gas = gas_df[(gas_df['Change'] > 0) & (gas_df['State'] != 'Total')]
    negative_changes_gas = gas_df[(gas_df['Change'] < 0) & (gas_df['State'] != 'Total')]

    positive_states_gas = [f"{row['State']} (+{row['Change']})" for _, row in positive_changes_gas.iterrows()]
    negative_states_gas = [f"{row['State']} ({row['Change']})" for _, row in negative_changes_gas.iterrows()]

    positive_total_gas = positive_changes_gas['Change'].sum()
    negative_total_gas = negative_changes_gas['Change'].sum()

    # Format the lists for positive and negative states
    if positive_states_gas:
        if len(positive_states_gas) == 1:
            positive_states_str_gas = positive_states_gas[0]
        elif len(positive_states_gas) == 2:
            positive_states_str_gas = f"{positive_states_gas[0]} and {positive_states_gas[1]}"
        else:
            positive_states_str_gas = ', '.join(positive_states_gas[:-1]) + ', and ' + positive_states_gas[-1]
    else:
        positive_states_str_gas = ''

    if negative_states_gas:
        if len(negative_states_gas) == 1:
            negative_states_str_gas = negative_states_gas[0]
        elif len(negative_states_gas) == 2:
            negative_states_str_gas = f"{negative_states_gas[0]} and {negative_states_gas[1]}"
        else:
            negative_states_str_gas = ', '.join(negative_states_gas[:-1]) + ', and ' + negative_states_gas[-1]
    else:
        negative_states_str_gas = ''

    # Construct the gas commentary
    if positive_states_gas and negative_states_gas:
        gas_paragraph = (
            f"The latest ({latest_date}) Baker Hughes natural gas-directed rig count {gas_change_phrase}, week-over-week (WoW). "
            f"{positive_states_str_gas} added {positive_total_gas} gas rig{'s' if positive_total_gas > 1 else ''}, while {negative_states_str_gas} dropped {abs(negative_total_gas)} gas rig{'s' if abs(negative_total_gas) > 1 else ''} this week."
            f" The Baker Hughes survey also indicated a total oil-directed rig count of {oil_total_latest}, {oil_change_phrase}."
        )
    elif positive_states_gas:
        gas_paragraph = (
            f"The latest ({latest_date}) Baker Hughes natural gas-directed rig count {gas_change_phrase}, week-over-week (WoW). "
            f"There were no rig losses reported, with {positive_states_str_gas} adding {positive_total_gas} gas rig{'s' if positive_total_gas > 1 else ''}."
            f" The Baker Hughes survey also indicated a total oil-directed rig count of {oil_total_latest}, {oil_change_phrase}."
        )
    else:
        gas_paragraph = (
            f"The latest ({latest_date}) Baker Hughes natural gas-directed rig count {gas_change_phrase}, week-over-week (WoW). "
            f"{negative_states_str_gas} dropped {abs(negative_total_gas)} gas rig{'s' if abs(negative_total_gas) > 1 else ''} this week."
            f" The Baker Hughes survey also indicated a total oil-directed rig count of {oil_total_latest}, {oil_change_phrase}."
        )

    return f"Weekly Commentary:\n\n{gas_paragraph}"

# Usage:
# Assuming formatted_dates, gas_df, and oil_df are already defined.
paragraph = generate_paragraph(formatted_dates, gas_df, oil_df)
print(paragraph)

Weekly Commentary:

The latest (Sep-06) Baker Hughes natural gas-directed rig count stood at 94, reflecting a decrease of 1 rig, week-over-week (WoW). West Virginia (+2) added 2 gas rigs, while Pennsylvania (-2) and Texas (-1) dropped 3 gas rigs this week. The Baker Hughes survey also indicated a total oil-directed rig count of 483, flat, WoW.
